## GD_NLP_04_reuters_classifiaction (DTM, TF-IDF Vocabulary Size)


Google에서 제공하는 오픈소스 기반 Sentence Tokenizer/Detokenizer 패키지인 SentencePiece를 사용하는 프로젝트입니다.



#### 프로젝트 설명
1. reuters 뉴스 데이터를 불러온다. (이 때 단어의 수를 조절할 수 있다.)
 - 전체 단어
 - 5000개 단어
 - 그 외로 실험해보기(3가지)
2. 머신러닝 모델 학습
 - 나이브 베이즈 분류기
 - CNB
 - 로지스틱 회귀
 - 서포트 벡터 머신(SVM)
 - 결정 트리
 - 랜덤 포레스트
 - 그래디언트 부스팅 트리
 - 보팅

In [1]:
# 모든 프로젝트에 해당되는 패키지를 먼저 임포트
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix